In [1]:
# 기본
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

# KFold (교차 검증을 사용하기 위해)
from sklearn.model_selection import KFold
from sklearn.model_selection import StratifiedKFold

# 교차검증 함수
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import cross_validate

# 데이터 전처리
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import StandardScaler

# 하이퍼 파라미터 튜닝
from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV

# 평가 함수
from sklearn.metrics import accuracy_score

# 머신러닝 알고리즘 - 분류
from sklearn.neighbors import KNeighborsClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import GradientBoostingClassifier
from xgboost import XGBClassifier

# 머신러닝 알고리즘 - 회귀
from sklearn.neighbors import KNeighborsRegressor
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import Ridge
from sklearn.linear_model import Lasso
from sklearn.linear_model import ElasticNet
from sklearn.svm import SVR
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.ensemble import GradientBoostingRegressor
from xgboost import XGBRFRegressor

# 그래프 설정
plt.rcParams['font.family'] = 'Malgun Gothic'   # 윈도우용
# plt.rcParams['font.family'] = 'AppleGothic'   # 맥용
plt.rcParams['font.size'] = 12                  # 폰트 크기
plt.rcParams['figure.figsize'] = 10,8          # 그래프 크기
plt.rcParams['axes.unicode_minus'] = False     # - 기호 깨짐 방지


# 경고 메시지가 안나오게 
import warnings
warnings.filterwarnings('ignore')

평균임금빼고 전부 결측치는 모름/무응답으로 
대체

평균임금을 결과 데이터로
평균임금의 결측치가 아닌 사람의 데이터를 예측해서 ..? 에?

### 데이터 불러오기

In [2]:
df = pd.read_csv('df_all.csv')
df.drop('Unnamed: 0',axis=1,inplace=True)
df

,성별,출생년도,혼인상태,종교유무,직종,평균임금,지역
0,2,1936,2,2,NaN,NaN,1
1,2,1945,2,2,NaN,NaN,1
2,1,1948,2,2,NaN,NaN,1
3,1,1942,3,1,762.0,108.9,1
4,2,1923,2,1,NaN,NaN,1
...,...,...,...,...,...,...,...
15417,2,1967,1,1,952.0,NaN,5
15418,2,1992,5,1,NaN,NaN,5
15419,1,1995,5,1,521.0,72.0,5
15420,2,1998,5,1,432.0,NaN,5


### 결측치 확인

In [3]:
df.isna().sum()

성별          0
출생년도        0
혼인상태        0
종교유무        0
직종       8291
평균임금    10915
지역          0
dtype: int64

In [4]:
# 직종 결측치 9999(모름/무응답)로 채우기
df['직종'].fillna(9999,inplace=True)

### 학습할 데이터 전처리
- 1. 평균임금이 Nan인 데이터 제거후 학습
- 2. 모두 포함하여 학습 (Nan값이 존재하여 학습이 안됨)

#### 1. 평균임금이 Nan인 데이터 제거후 학습

In [5]:
df_1 = df.copy()
df_1.head()

,성별,출생년도,혼인상태,종교유무,직종,평균임금,지역
0,2,1936,2,2,9999.0,NaN,1
1,2,1945,2,2,9999.0,NaN,1
2,1,1948,2,2,9999.0,NaN,1
3,1,1942,3,1,762.0,108.9,1
4,2,1923,2,1,9999.0,NaN,1


In [6]:
df_1.dropna(inplace=True)

In [7]:
x_data = df_1.drop('평균임금', axis=1)
y_data = df_1['평균임금']

In [8]:
# 데이터 나누기
x_train, x_test, y_train, y_test=train_test_split(x_data, y_data, test_size=0.2, random_state=1)

In [9]:
fold1=KFold(n_splits=10, shuffle=True, random_state=1)

#### KNN

In [10]:
params ={
    'n_neighbors' : [1,2,3,4,5,6,7,8,9,10]
}

model1 = KNeighborsRegressor()
grid1 = GridSearchCV(model1, param_grid=params, scoring='r2', cv=fold1)
grid1.fit(x_train,y_train)
print(f'최적의 하이퍼 파라미터 : {grid1.best_params_}')
print(f'최적의 모델 평균 성능 : {grid1.best_score_}') # 정확도는 아님 (1에 가까울수록 좋음)

최적의 하이퍼 파라미터 : {'n_neighbors': 8}
최적의 모델 평균 성능 : 0.4171210594783825


In [11]:
best1 = grid1.best_estimator_

In [12]:
print(best1)

KNeighborsRegressor(algorithm='auto', leaf_size=30, metric='minkowski',
                    metric_params=None, n_jobs=None, n_neighbors=8, p=2,
                    weights='uniform')


In [13]:
# 교차 검증
r1 = cross_val_score(best1, x_train,y_train, scoring='r2', cv=fold1)
print(r1.mean())

0.4171210594783825


In [14]:
# 모델에 학습
best1.fit(x_train, y_train)

KNeighborsRegressor(algorithm='auto', leaf_size=30, metric='minkowski',
                    metric_params=None, n_jobs=None, n_neighbors=8, p=2,
                    weights='uniform')

In [15]:
pred1 = best1.predict(x_test)
pred1

array([ 40.75   , 109.475  , 345.5    , 132.0125 , 559.725  , 333.25   ,
       354.4125 , 417.25   , 357.9625 , 305.     , 420.     ,  73.75   ,
       183.5    , 269.5    , 102.375  , 490.875  , 134.75   ,  82.375  ,
       330.     , 264.375  , 173.375  , 238.375  , 135.375  , 441.25   ,
       216.5    , 129.875  , 143.25   , 345.5    , 528.7    , 457.125  ,
       175.0875 , 223.25   , 189.875  ,  94.625  , 162.125  , 119.35   ,
       139.875  , 448.125  , 123.25   , 372.125  , 168.25   , 409.125  ,
       109.475  , 393.     , 177.25   , 183.     , 151.85   , 267.125  ,
       322.25   ,  21.25   , 459.75   , 444.875  , 416.0875 , 449.875  ,
       546.7875 , 171.875  , 371.     , 523.6625 , 136.6875 , 128.3    ,
       152.     ,  71.     , 482.975  ,  93.125  , 165.     , 285.875  ,
       408.4125 ,  82.25   , 244.575  , 354.25   , 162.125  , 446.25   ,
       403.625  , 304.875  , 113.625  , 316.2875 , 196.4125 , 491.375  ,
       154.75   , 223.45   , 158.125  , 561.1375 , 

In [16]:
best1.score(x_test,y_test)

0.394348751993219

#### Ridge

In [17]:
# Ridge

params = {
    # 값이 낮으면 규제가 약해지고 값이 크면 규제가 강해진다
    'alpha' : [0.0001, 0.001, 0.01, 0.1, 1, 10, 100, 1000, 1000]
}

model2 = Ridge()
grid2 = GridSearchCV(model2, param_grid=params, scoring='r2', cv=fold1)
grid2.fit(x_train,y_train)
print(f'최적의 하이퍼 파라미터 : {grid2.best_params_}')
print(f'최적의 모델 평균 성능 : {grid2.best_score_}')


최적의 하이퍼 파라미터 : {'alpha': 10}
최적의 모델 평균 성능 : 0.34274900818111886


#### Lasso

In [18]:
# Lasso

params = {
    # 값이 낮으면 규제가 약해지고 값이 크면 규제가 강해진다
    'alpha' : [0.0001, 0.001, 0.01, 0.1, 1, 10, 100, 1000, 1000]
}

model3 = Lasso()
grid3 = GridSearchCV(model3, param_grid=params, scoring='r2', cv=fold1)
grid3.fit(x_train,y_train)
print(f'최적의 하이퍼 파라미터 : {grid3.best_params_}')
print(f'최적의 모델 평균 성능 : {grid3.best_score_}')

최적의 하이퍼 파라미터 : {'alpha': 0.1}
최적의 모델 평균 성능 : 0.3427281444534969


#### ElasticNet

In [19]:
# ElasticNet

params = {
    # 값이 낮으면 규제가 약해지고 값이 크면 규제가 강해진다
    'alpha' : [0.0001, 0.001, 0.01, 0.1, 1, 10, 100, 1000, 1000]
}

model4 = ElasticNet()
grid4 = GridSearchCV(model4, param_grid=params, scoring='r2', cv=fold1)
grid4.fit(x_train,y_train)
print(f'최적의 하이퍼 파라미터 : {grid4.best_params_}')
print(f'최적의 모델 평균 성능 : {grid4.best_score_}')

최적의 하이퍼 파라미터 : {'alpha': 0.01}
최적의 모델 평균 성능 : 0.3427401670839959


### KNN 모델 선정 값 예측하기

In [37]:
x_data_all = df.drop('평균임금', axis=1)
y_data_all = df['평균임금']

In [39]:
pred100 = best1.predict(x_data_all)
pred100

array([515.2625, 515.2625, 515.2625, ..., 124.    , 173.375 , 338.1125])

In [42]:
x_data_all['예측평균임금'] = pred100
x_data_all['예측평균임금'] = x_data_all['예측평균임금'].round(decimals=2)

In [43]:
x_data_all

,성별,출생년도,혼인상태,종교유무,직종,지역,예측평균임금
0,2,1936,2,2,9999.0,1,515.26
1,2,1945,2,2,9999.0,1,515.26
2,1,1948,2,2,9999.0,1,515.26
3,1,1942,3,1,762.0,1,222.49
4,2,1923,2,1,9999.0,1,515.26
...,...,...,...,...,...,...,...
15417,2,1967,1,1,952.0,5,133.25
15418,2,1992,5,1,9999.0,5,338.11
15419,1,1995,5,1,521.0,5,124.00
15420,2,1998,5,1,432.0,5,173.38
